In [1]:
!pip install pytorch-lightning
!pip install wandb 
import wandb

wandb.init(project="Data-reordering-attacks-test-100epochs-2", entity="saurav07")

wandb: Currently logged in as: saurav07 (use `wandb login --relogin` to force relogin)


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import random
import torchvision.transforms.functional as TF


import pytorch_lightning as pl
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torchmetrics.functional import accuracy
from torch.optim import Adam
from torch.utils.data import Sampler


import numpy as np
from sklearn.metrics import classification_report
from collections import OrderedDict
from typing import Sized, Iterator

In [3]:
class Lenet5(nn.Module):
    
    def __init__(self)->None:
        super(Lenet5, self).__init__()
        
        self.layer1 = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(3, 6, kernel_size=(5, 5))),
            ('relu1', nn.ReLU()),
            ('pool1', nn.MaxPool2d(kernel_size=(2, 2), stride=2))
        ]))
        
        self.layer2 = nn.Sequential(OrderedDict([
            ('conv2', nn.Conv2d(6, 16, kernel_size=(5, 5))),
            ('relu2', nn.ReLU()),
            ('pool2', nn.MaxPool2d(kernel_size=(2, 2), stride=2))
        ]))
        
        self.fc1 = nn.Sequential(OrderedDict([
            ('f4', nn.Linear(400, 84)),
            ('relu4', nn.ReLU())
        ]))
        
        
        self.fc2 = nn.Sequential(OrderedDict([
            ('f5', nn.Linear(84, 10)),
        ]))

        
    def forward(self, x):
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.shape[0], -1)
        
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [5]:
surrogate = Lenet5()
surrogate.to(device)

Lenet5(
  (layer1): Sequential(
    (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (relu1): ReLU()
    (pool1): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (relu2): ReLU()
    (pool2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Sequential(
    (f4): Linear(in_features=400, out_features=84, bias=True)
    (relu4): ReLU()
  )
  (fc2): Sequential(
    (f5): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [15]:
class BatchReorderSampler(Sampler[int]):

    def __init__(self, data_source: Sized, atk_type="lowtohigh", surrogate= surrogate, batch_size=32) -> None:
        self.data_source = data_source
        self.surrogate = surrogate

        self.epoch1 = True
        self.backdoor = False
        
        self.batchOrder = torch.randperm((len(data_source)//batch_size)*batch_size)
        temp = np.array(list(range(self.batchOrder.shape[0])))
        self.batchOrder = torch.from_numpy(temp)
        self.batchOrder = self.batchOrder.reshape(-1, batch_size)    
        print("batch = ", self.batchOrder[:5])    
        
        data = [self.data_source.__getitem__(j) for j in self.batchOrder.view(-1)]
        data, labels = zip(*data)
        self.data = torch.stack(data).to(device)
        self.labels = torch.LongTensor(labels).to(device)
        self.atk_type = atk_type
     
    def __getSurrogateloss__(self, batch):
        
        with torch.no_grad():
            loss = F.nll_loss(self.surrogate(self.data[batch]) ,self.labels[batch])
        return loss.cpu().item()
    
    
    def __iter__(self) -> Iterator[int]:
        
        
        if self.epoch1 == True:
            print('Waiting to Attack')
            for i in range(self.batchOrder.shape[0]):
                yield iter(self.batchOrder[i])
            self.epoch1 = False
        else:

            print('Attacking')
            print("batch order: ", self.batchOrder)
            losses = torch.Tensor([self.__getSurrogateloss__(batch) for batch in self.batchOrder])

            if self.atk_type == "lowtohigh":
              for i in torch.argsort(losses):
                yield iter(self.batchOrder[i])

            elif self.atk_type == "hightolow":
              for i in torch.argsort(losses, descending=True):
                  yield iter(self.batchOrder[i])

            elif self.atk_type == "oscillating in":
              asc_sort_losses = torch.argsort(losses).tolist()
              oscillate_inward_sort = []
              asc_n = len(asc_sort_losses) - 1
              for loss_ind in range(asc_n//2 + 1):
                oscillate_inward_sort.append(asc_sort_losses[asc_n - loss_ind])
                oscillate_inward_sort.append(asc_sort_losses[loss_ind])
              oscillate_inward_sort_tensor = torch.tensor(np.array(oscillate_inward_sort))
              for i in oscillate_inward_sort_tensor:
                   yield iter(self.batchOrder[i])

            elif self.atk_type == "oscillating out":
              asc_sort_losses = torch.argsort(losses).tolist()
              oscillate_outward_sort = []
              asc_n = len(asc_sort_losses) - 1
              for loss_ind in range(asc_n//2 + 1):
                oscillate_outward_sort.append(asc_sort_losses[asc_n - loss_ind])
                oscillate_outward_sort.append(asc_sort_losses[loss_ind])
              oscillate_outward_sort_tensor = torch.tensor(np.array(oscillate_outward_sort))
              oscillate_outward_sort_tensor = torch.flip(oscillate_outward_sort_tensor, [0])
              for i in oscillate_outward_sort_tensor:
                  yield iter(self.batchOrder[i])
            
    def __len__(self) -> int:
        return self.batchOrder.shape[0]

In [7]:
# temp_torch_arr = torch.tensor(np.array([1, 2, 3,4, 5, 6]))
# # temp_torch_arr = temp_torch_arr.view(-1, 2)
# torch.flip(temp_torch_arr, [0])

BATCH_ORDER = None

In [8]:
class BatchShuffleSampler(Sampler[int]):

    def __init__(self, data_source: Sized,  atk_type="lowtohigh", surrogate=surrogate, batch_size=32) -> None:
        self.data_source = data_source
        
        self.surrogate = surrogate
        self.batch_size = batch_size

        self.epoch1 = True
        self.batchOrder = torch.randperm((len(data_source)//batch_size)*batch_size)
        temp = np.array(list(range(self.batchOrder.shape[0])))
        self.batchOrder = torch.from_numpy(temp)
        print("batch order shape = ", self.batchOrder.shape)
        print("data source length = ", len(data_source))
        print("batch order = ", self.batchOrder[:10])
        
        data = [self.data_source.__getitem__(j) for j in self.batchOrder.view(-1)]
        data, labels = zip(*data)
        self.data = torch.stack(data).to(device)
        self.labels = torch.LongTensor(labels).to(device)

        self.atk_type = atk_type
     
    def __getSurrogateloss__(self, batch):
        
        
        with torch.no_grad():
            # random_batch_perm = torch.randperm(batch_size)
            # data_perm = self.data[random_batch_perm]
            # label_perm = self.labels[random_batch_perm]
            
            loss = F.nll_loss(self.surrogate(self.data[batch: batch+1]) 
            ,self.labels[batch: batch+1])

        return loss.cpu().item()
    
    
    def __iter__(self) -> Iterator[int]:
        
        if self.epoch1 == True:
            print('Waiting to Attack')
            print(self.batchOrder.view(-1, self.batch_size)[:2])
            for i in self.batchOrder.view(-1, self.batch_size):
                yield iter(i)
            self.epoch1 = False
            
        else:
            print('Attacking')
            print("batch order: ",self.batchOrder)

            losses = torch.Tensor([self.__getSurrogateloss__(batch) for batch in self.batchOrder])
            print("printing losses now _______")
            print(losses[:5])
            print(torch.argsort(losses)[:5])
            print(losses[torch.argsort(losses)[:5]])
            print("printing losses done _______")
            
            if self.atk_type == "lowtohigh":
              # print("low to high running")
              print(self.batchOrder[torch.argsort(losses)].view(-1, self.batch_size)[:2])
              for i in self.batchOrder[torch.argsort(losses)].view(-1, self.batch_size):
                  yield iter(i)

            elif self.atk_type == "hightolow":
              print("high to low running")
              print(self.batchOrder.shape)
              print(self.batchOrder[torch.argsort(losses, descending=True)].view(-1, self.batch_size).shape)
              for i in self.batchOrder[torch.argsort(losses, descending=True)].view(-1, self.batch_size):
                  yield iter(i)

            elif self.atk_type == "oscillating in":
              print("oscillating in running")
              
              asc_sort_losses = torch.argsort(losses).tolist()

              oscillate_inward_sort = []
              asc_n = len(asc_sort_losses) - 1
              for loss_ind in range(asc_n//2 + 1):
                oscillate_inward_sort.append(asc_sort_losses[asc_n - loss_ind])
                oscillate_inward_sort.append(asc_sort_losses[loss_ind])
              oscillate_inward_sort_tensor = torch.tensor(np.array(oscillate_inward_sort))

              for i in self.batchOrder[oscillate_inward_sort_tensor].view(-1, self.batch_size):
                  yield iter(i)
            
            elif self.atk_type == "oscillating out":
              print("oscillating out running")
              asc_sort_losses = torch.argsort(losses).tolist()
              oscillate_outward_sort = []
              asc_n = len(asc_sort_losses) - 1
              for loss_ind in range(asc_n//2 + 1):
                oscillate_outward_sort.append(asc_sort_losses[asc_n - loss_ind])
                oscillate_outward_sort.append(asc_sort_losses[loss_ind])
              oscillate_outward_sort_tensor = torch.tensor(np.array(oscillate_outward_sort))
              oscillate_outward_sort_tensor = torch.flip(oscillate_outward_sort_tensor, [0])
              for i in self.batchOrder[oscillate_outward_sort_tensor].view(-1, self.batch_size):
                  yield iter(i)
            
            print('Attack successful')
        

    def __len__(self) -> int:
        return self.batchOrder.view(-1, self.batch_size).shape[0]

In [16]:
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_transform = transforms.Compose(
    [
     torchvision.transforms.RandomCrop(32, padding=4),
#      torchvision.transforms.Resize(),
     torchvision.transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

attack_type = ["lowtohigh", "hightolow", "oscillating in", "oscillating out"]

batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)

# trainloader = torch.utils.data.DataLoader(trainset, num_workers=16, 
#                                           batch_sampler=BatchShuffleSampler(trainset, atk_type=attack_type[0]))
trainloader = torch.utils.data.DataLoader(trainset, num_workers=16, 
                                          batch_sampler=BatchReorderSampler(trainset, atk_type=attack_type[0]))
# trainloader = torch.utils.data.DataLoader(trainset, 
#                                           num_workers=16, batch_sampler=BatchShuffleSampler(trainset, attack_type[3]))


testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=16)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
batch =  tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
          28,  29,  30,  31],
        [ 32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,
          46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,
          60,  61,  62,  63],
        [ 64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
          78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
          92,  93,  94,  95],
        [ 96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
         110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123,
         124, 125, 126, 127],
        [128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
         142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
         156, 157, 158, 159]])


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


In [11]:

def resnet18CIFAR10():
    model = torchvision.models.resnet18(pretrained=False, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    model.maxpool = nn.Identity()
    
    return model


class BoilerPlate(pl.LightningModule):
    def __init__(self, train_l, val_l, surrogate) -> None:
        super(BoilerPlate, self).__init__()

        self.train_l = train_l
        self.val_l = val_l
        
        self.model = resnet18CIFAR10()
        
        
        self.surrogate = surrogate
        self.surrogate_optim = Adam([p for p in self.surrogate.parameters() if p.requires_grad], lr=0.1)
        

    def forward(self, x):
        out = self.model(x)
        
        return F.log_softmax(out, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)        
        loss = F.nll_loss(logits, y) 

        wandb.log({"model loss":loss})
        
        self.surrogate_optim.zero_grad()
        surrogate_logits = self.surrogate(x)
        surrogateloss = F.nll_loss(surrogate_logits, y)
        wandb.log({"surrogate loss" : surrogateloss})
        surrogateloss.backward()
        self.surrogate_optim.step()
        # print("loss = ", loss)
        
        return loss
    

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        # self.log('loss', loss.item(), on_step=True)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        if stage:
            wandb.log({f"{stage}_acc" : acc})
            wandb.log({f"{stage}_loss": loss})
            self.log(f"{stage}_loss", loss, prog_bar=True, on_step = True)
            self.log(f"{stage}_acc", acc, prog_bar=True, on_step = True)

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    # def validation_epoch_end(self, outputs):
    #     avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
    #     self.logger.experiment.add_scalar('loss',avg_loss, self.current_epoch)



    def configure_optimizers(self):
        return Adam([p for p in self.model.parameters() if p.requires_grad], lr=0.1)

    def train_dataloader(self):
        return self.train_l

    def val_dataloader(self):
        return self.val_l
    
    # def validation_step(self, batch: Tuple[Tensor, Tensor], _batch_index: int) -> None:
    #   inputs_batch, labels_batch = batch

    #   outputs_batch = self(inputs_batch)
    #   loss = self.criterion(outputs_batch, labels_batch)

    #   self.log('loss', loss.item(), on_step=True)

In [17]:
from pytorch_lightning.loggers import WandbLogger
model = BoilerPlate(trainloader, testloader, surrogate)

wandb.init(project="Data-ordering-attacks-test-100epochs-2", entity="saurav07")

surrogate = Lenet5()
# surrogate = resnet18CIFAR10()
surrogate.to(device)



trainer = pl.Trainer(
    progress_bar_refresh_rate=100,
    max_epochs=10,
    gpus=1,
    logger=pl.loggers.TensorBoardLogger("lightning_logs/", name="test"),
    # logger = wandb_logger
)
trainer.fit(model)

val_acc,▁█
val_loss,▁█
val_acc,0.10938
val_loss,2.43612


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=100)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type   | Params
-------------------------------------
0 | model     | ResNet | 11.2 M
1 | surrogate | Lenet5 | 37.4 K
-------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total param

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training: 0it [00:00, ?it/s]

Waiting to Attack


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


batch_size = 32
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=16)

model.to(device)
model.eval()
preds, labels = [], []
for batch in testloader:
    x, y = batch
    x = x.to(device)
    logits = model(x)
    y_pred = torch.argmax(logits, dim=1)
    
    preds.append(y_pred.cpu().numpy())
    labels.append(y.cpu().numpy())

preds = np.concatenate(preds)
labels = np.concatenate(labels)
print(classification_report(labels, preds, target_names=classes))